## Evaluate PF-controller neural network

**Note:** This evaluation follows a leave-one-unit-out protocol applied independently to each performance metric. For a given evaluation unit and performance metric, the particle filter and its neural transition model are inferred using data from all remaining units.

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import imageio
import matplotlib.pyplot as plt


from src.models.particle_filter.core import ParticleFilterMLP, ParticleFilterModel
from src.helpers.visualization import create_pf_prediciton_frame
from src.helpers.seed import set_global_seed


from experiment_config import (
    DegModel,DATA_NAME,FILTERED_SUFFIX,SEED,PFNET_NAME,LEAKY_SLOPE,HIDDEN_DIMS,PREDICTION_START_IDX
)

## Parameters

In [2]:
perform_name = "T48"

n_particles = 1800

In [3]:
degmodel_name = DegModel.name()
activation = nn.LeakyReLU(LEAKY_SLOPE)
set_global_seed(SEED)
experiment_dir = Path('experiments')/DATA_NAME
degmodel_dir = experiment_dir/f'degradation{FILTERED_SUFFIX}'/degmodel_name
pfnet_dir = degmodel_dir /'pf_performs'/PFNET_NAME

## Import development data

In [4]:
hi_df = pd.read_csv(experiment_dir/f'hidata_dev{FILTERED_SUFFIX}.csv')
del hi_df['hs']
units = hi_df['unit'].astype(int).unique().tolist()
units 

[1, 2, 3, 4, 5, 6]

In [5]:
perform_names = [col for col in hi_df.columns if col not in ['unit','cycle']]

performs = {name: 
    {unit: hi_df[hi_df['unit']==unit][name].values for unit in units} 
    for name in perform_names
}
time = {unit: hi_df[hi_df['unit']==unit]['cycle'].values for unit in units}

## Prepare data

In [6]:
eval_data = {}
for unit in units:
    t_data = time[unit]
    s_data = performs[perform_name][unit]
    eval_data[unit]=torch.tensor(np.stack([t_data, s_data],axis=1),dtype=torch.float32)


## Load component (base) models

In [7]:
train_degmodels = {}
for eval_unit in units:
    degmodels=[]
    for unit, perform in performs[perform_name].items():
        if unit == eval_unit:
            continue
        best_model = DegModel()
        best_model.load_state_dict(
            torch.load(degmodel_dir/'states'/perform_name/f'unit_{unit}'/ "best_model.pt")
        )
        degmodels.append(best_model)
    train_degmodels[eval_unit] = degmodels
    

## Load PF-net

In [8]:
ckpt = torch.load(pfnet_dir/ perform_name / "checkpoint.pt",weights_only=False)
net = ParticleFilterMLP(state_dim=DegModel.state_dim(), hidden_dims=HIDDEN_DIMS,
                        activation=lambda : activation)
net.load_state_dict(ckpt['model_state'])

net.eval()  

ParticleFilterMLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.05)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.05)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LeakyReLU(negative_slope=0.05)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.05)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): LeakyReLU(negative_slope=0.05)
    (10): Linear(in_features=32, out_features=16, bias=True)
  )
)

## Run Particle Filter with (leraned net)

Save video

In [9]:

t_grid = np.linspace(0.1, 100, 80)
s_grid = np.linspace(0.0, 1.0, 60)
conf_level = 0.95

for eval_unit in units:
    degmodels = train_degmodels[eval_unit]
    t_data = eval_data[eval_unit][:, 0]
    s_data = eval_data[eval_unit][:, 1]
    t_data_np = t_data.cpu().numpy()
    s_data_np = s_data.cpu().numpy()

    pf = ParticleFilterModel(
        base_models=degmodels,
        net=net,
        n_particles=n_particles,
    ).eval()

    frames = []
    for k in range(PREDICTION_START_IDX, len(t_data)):
        pf.step(
            t_obs=t_data[:k+1],
            s_obs=s_data[:k+1],
        )
        
        lower, mean, upper = pf.mixture.uncertainty_interval(s=torch.zeros(1), level=conf_level)
        # --- render frame ---
        fig, ax = plt.subplots(figsize=(10, 8))
        
        create_pf_prediciton_frame(
            ax,
            pf,
            t_grid,
            s_grid,
            t_data_np,
            s_data_np,
            pred_interval=(lower.item(), mean.item(), upper.item()),
            conf_level=conf_level,
            current_step = k,
            dist_vmax=0.25,
            dist_plot_mean=True,
        )
        # create frame
        fig.canvas.draw()
        frame = np.asarray(fig.canvas.renderer.buffer_rgba())
        plt.close(fig)
        frames.append(frame)
  
    ## Save video
    video_path = pfnet_dir /perform_name/ f"pf_eval{eval_unit}_cap.mp4"

    with imageio.get_writer(video_path, fps=8, macro_block_size=1) as writer:
        for frame in frames:
            writer.append_data(frame)

    print(f"🎬 Video saved to {video_path}")


🎬 Video saved to experiments/DS01/degradation_filtered/normal/pf_performs/net256x256x128x64x32leaky0.05/T48/pf_eval1_cap.mp4
🎬 Video saved to experiments/DS01/degradation_filtered/normal/pf_performs/net256x256x128x64x32leaky0.05/T48/pf_eval2_cap.mp4
🎬 Video saved to experiments/DS01/degradation_filtered/normal/pf_performs/net256x256x128x64x32leaky0.05/T48/pf_eval3_cap.mp4
🎬 Video saved to experiments/DS01/degradation_filtered/normal/pf_performs/net256x256x128x64x32leaky0.05/T48/pf_eval4_cap.mp4
🎬 Video saved to experiments/DS01/degradation_filtered/normal/pf_performs/net256x256x128x64x32leaky0.05/T48/pf_eval5_cap.mp4
🎬 Video saved to experiments/DS01/degradation_filtered/normal/pf_performs/net256x256x128x64x32leaky0.05/T48/pf_eval6_cap.mp4
